In [ ]:
#!pip install filterpy

In [ ]:
import numpy as np
from filterpy.kalman import KalmanFilter
import matplotlib.pyplot as plt

# Создаем фильтр Калмана
kf = KalmanFilter(dim_x=2, dim_z=1)

# Матрицы состояния
kf.F = np.array([[1, 1], [0, 1]])  # Переходная матрица (x_t = A x_{t-1})
kf.H = np.array([[1, 0]])  # Связь измерений с состоянием
kf.P *= 100  # Начальная неопределенность
kf.R = 50  # Шум измерений
kf.Q = np.array([[1, 0], [0, 1]])  # Шум процесса

# Исходные данные (пример: истинный тренд + шум)
np.random.seed(42)
timesteps = 50
true_values = np.linspace(0, 10, timesteps)  # Линейный тренд
measurements = true_values + np.random.normal(0, 1, timesteps)  # Добавляем шум

# Запускаем фильтр
filtered_values = []
kf.x = np.array([[0], [0]])  # Начальное состояние

for z in measurements:
    kf.predict()
    kf.update([z])
    filtered_values.append(kf.x[0, 0])  # Берем первое состояние (позицию)

# Визуализация
plt.plot(measurements, label="Шумные измерения", linestyle="dotted", alpha=0.5)
plt.plot(true_values, label="Истинные значения", linewidth=2)
plt.plot(filtered_values, label="Фильтр Калмана", linewidth=2)
plt.legend()
plt.show()


In [ ]:
#!pip install statsmodels

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Создаем данные с шумом
# np.random.seed(42)
# t = np.arange(100)
# y = np.sin(t / 10) + np.random.normal(0, 0.1, len(t))  # Исходный ряд с шумом

# Применяем HP-фильтр
lmbda = 1600  # Можно попробовать другие значения
cycle, trend = sm.tsa.filters.hpfilter(y, lamb=lmbda)

# Визуализация
plt.plot(t, y, label="Исходные данные", alpha=0.5)
plt.plot(t, trend, label="HP-фильтр (тренд)", linewidth=2)
plt.legend()
plt.show()


In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Создаем данные с шумом
np.random.seed(42)
t = np.arange(100)
y = np.sin(t / 10) + np.random.normal(0, 0.1, len(t))  # Исходный ряд с шумом

# Применяем HP-фильтр
lmbda = 16  # Можно попробовать другие значения
cycle, trend = sm.tsa.filters.hpfilter(measurements, lamb=lmbda)

# Визуализация
# plt.plot(t, y, label="Исходные данные", alpha=0.5)
# plt.plot(t, trend, label="HP-фильтр (тренд)", linewidth=2)
# plt.legend()
# plt.show()

plt.plot(measurements, label="Шумные измерения", linestyle="dotted", alpha=0.5)
plt.plot(true_values, label="Истинные значения", linewidth=2)
plt.plot(trend, label="trend", linewidth=2)
plt.plot(cycle, label="cycle", linewidth=2)
plt.legend()
plt.show()


In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Создаем данные с шумом
np.random.seed(42)
t = np.arange(100)
y = np.sin(t / 10) + np.random.normal(0, 0.1, len(t))  # Исходный ряд с шумом

# Применяем HP-фильтр
lmbda = 10  # Можно попробовать другие значения
cycle, trend = sm.tsa.filters.hpfilter(y, lamb=lmbda)

# Визуализация
plt.plot(t, y, label="Исходные данные", alpha=0.5)
plt.plot(t, trend, label="HP-фильтр (тренд)", linewidth=2)
plt.legend()
plt.show()


In [ ]:
import numpy as np
import scipy.integrate
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Параметры Лоренца
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Функция системы Лоренца
def lorenz_system(state, t):
    x, y, z = state
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return [dx, dy, dz]

# Генерация данных
t = np.linspace(0, 10, 1000)  # Временная шкала
initial_state = [1.0, 1.0, 1.0]  # Начальные условия
solution = scipy.integrate.odeint(lorenz_system, initial_state, t)

x = solution[:, 2]  # Берем x-компоненту
x_noisy = x + np.random.normal(0, 0.01, len(x))  # Добавляем шум

# Применяем HP-фильтр
lambda_hp = 1600  # Можно варьировать
x_smooth, _ = sm.tsa.filters.hpfilter(x_noisy, lamb=lambda_hp)

# Вычисление первой и второй производных
dx_dt = np.zeros_like(x_smooth)
d2x_dt2 = np.zeros_like(x_smooth)

dx_dt[1:-1] = (x_smooth[2:] - x_smooth[:-2]) / (t[2:] - t[:-2])
d2x_dt2[1:-1] = (x_smooth[2:] - 2 * x_smooth[1:-1] + x_smooth[:-2]) / ((t[1] - t[0]) ** 2)

# Граничные условия
dx_dt[0] = (x_smooth[1] - x_smooth[0]) / (t[1] - t[0])
dx_dt[-1] = (x_smooth[-1] - x_smooth[-2]) / (t[-1] - t[-2])

d2x_dt2[0] = (x_smooth[2] - 2 * x_smooth[1] + x_smooth[0]) / ((t[1] - t[0]) ** 2)
d2x_dt2[-1] = (x_smooth[-1] - 2 * x_smooth[-2] + x_smooth[-3]) / ((t[1] - t[0]) ** 2)

# Визуализация
plt.figure(figsize=(12, 6))

plt.subplot(3, 1, 1)
plt.plot(t, x_noisy, label="Шумный x", alpha=0.5)
plt.plot(t, x_smooth, label="HP-фильтр x", linewidth=2)
plt.legend()

plt.subplot(3, 1, 2)
plt.plot(t, dx_dt, label="Первая производная (dx/dt)", color='orange')
plt.legend()

plt.subplot(3, 1, 3)
plt.plot(t, d2x_dt2, label="Вторая производная (d²x/dt²)", color='red')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Параметры системы Лоренца
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Уравнения Лоренца
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Начальные условия и время интегрирования
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 25)  # временной интервал
t_eval = np.linspace(t_span[0], t_span[1], 1000)  # временные точки

# Интеграция системы
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)
x = sol.y[0]
y = sol.y[1]
z = sol.y[2]

noise = 1
# Добавляем шум
x_noisy = x + np.random.normal(0, noise, len(x))
y_noisy = y + np.random.normal(0, noise, len(y))
z_noisy = z + np.random.normal(0, noise, len(z))

# Применяем HP-фильтр
lambda_hp = 16
_, x_smooth = sm.tsa.filters.hpfilter(x_noisy, lamb=lambda_hp)
_, y_smooth = sm.tsa.filters.hpfilter(y_noisy, lamb=lambda_hp)
_, z_smooth = sm.tsa.filters.hpfilter(z_noisy, lamb=lambda_hp)

# Визуализация
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

axes[0].plot(t_eval, x_noisy, label="Шумный x", alpha=0.5)
axes[0].plot(t_eval, x_smooth, label="HP-фильтр x", linewidth=2)
axes[0].legend()

axes[1].plot(t_eval, y_noisy, label="Шумный y", alpha=0.5)
axes[1].plot(t_eval, y_smooth, label="HP-фильтр y", linewidth=2)
axes[1].legend()

axes[2].plot(t_eval, z_noisy, label="Шумный z", alpha=0.5)
axes[2].plot(t_eval, z_smooth, label="HP-фильтр z", linewidth=2)
axes[2].legend()

plt.tight_layout()
plt.show()

# Фазовый портрет
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z, label="Оригинальный аттрактор", alpha=0.3)
ax.plot(x_noisy, y_noisy, z_noisy, label="Оригинальный аттрактор", alpha=0.3)
ax.plot(x_smooth, y_smooth, z_smooth, label="Сглаженный аттрактор", linewidth=1)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend()
plt.show()
